# Machine learning for Drug – Nanoparticle (DADNP) Systems in HCC


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# remove warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix,accuracy_score, roc_auc_score,f1_score, recall_score, precision_score
from sklearn.utils import class_weight

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import RFECV, VarianceThreshold, SelectKBest, chi2
from sklearn.feature_selection import SelectFromModel, SelectPercentile, f_classif

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

### Define script parameters

In [ ]:
# define output variables
outVars = ['f(vnj)obs06']

# define list of folds
foldTypes = [3]

# define a label for output files
targetName = 'sel'

seed = 42
np.random.seed(seed)

### Function definitions

In [ ]:
def  set_weights(y_data, option='balanced'):
    """Estimate class weights for umbalanced dataset
       If ‘balanced’, class weights will be given by n_samples / (n_classes * np.bincount(y)). 
       If a dictionary is given, keys are classes and values are corresponding class weights. 
       If None is given, the class weights will be uniform """
    cw = class_weight.compute_class_weight(option, np.unique(y_data), y_data)
    w = {i:j for i,j in zip(np.unique(y_data), cw)}
    return w 

Write a function to get data from the datafile:

In [ ]:
def getDataFromDataset(sFile, OutVar):
    # read details file
    print('\n-> Read dataset', sFile)
    df = pd.read_csv(sFile, sep='\t')
    print('Shape', df.shape)
    print(list(df.columns))

    # select X and Y
    ds_y = df[OutVar]
    ds_X = df.drop(OutVar,axis = 1)
    Xdata = ds_X.values # get values of features
    Ydata = ds_y.values # get output values

    print('Shape X data:', Xdata.shape)
    print('Shape Y data:',Ydata.shape)
    
    # return data for X and Y, feature names as list
    return (Xdata, Ydata, list(ds_X.columns))

Define a function to run all the classifiers:

In [ ]:
def Pipeline_OuterCV(Xdata, Ydata, label = 'my', class_weights = {0: 1, 1: 1}, folds = 3, seed = 42, metrics='roc_auc'):
    # inputs:
    # data for X, Y; a label about data, class_weights, number of folds, seeed
    
    # default: 10-fold CV, 1:1 class weights (balanced dataset)
    priors = [(class_weights[0]/(class_weights[0]+class_weights[1])), (class_weights[1]/(class_weights[0]+class_weights[1]))]
    
    # define classifiers
    names = ['KNN', 'GNB', 'LDA', 'LogR', 'MLP', 'DT', 'RF', 'XGB', 'GB', 'BAG', 'ADA']
    classifiers = [KNeighborsClassifier(n_jobs=-1),
               GaussianNB(),
               LinearDiscriminantAnalysis(solver='svd',priors=priors), # No tiene random_state
               LogisticRegression(solver='lbfgs',random_state=seed,class_weight=class_weights,max_iter=20000),
               MLPClassifier(hidden_layer_sizes= (5), random_state = seed, max_iter=50000, shuffle=False),
               DecisionTreeClassifier(random_state = seed,class_weight=class_weights),
               RandomForestClassifier(n_jobs=-1,random_state=seed,class_weight=class_weights),
               XGBClassifier(n_jobs=-1,seed=seed,scale_pos_weight= class_weights[0]/class_weights[1]),
               GradientBoostingClassifier(random_state=seed),
               BaggingClassifier(random_state=seed),
               AdaBoostClassifier(random_state = seed)]
    
    # results dataframe: each column for a classifier
    df_res = pd.DataFrame(columns=names)

    # build each classifier
    print('* Building scaling+feature selection+outer '+str(folds)+'-fold CV for '+str(len(names))+' classifiers:', str(names))
    total = time.time()
    
    # define a fold-CV for all the classifier
    outer_cv = StratifiedKFold(n_splits=folds,shuffle=True,random_state=seed)
    
    for name, clf in zip(names, classifiers):
        start = time.time()
        
        # create pipeline: scaler + classifier
        estimators = []
        
        # SCALER
        estimators.append(('Scaler', StandardScaler()))
        
        # add Classifier
        estimators.append(('Classifier', clf)) 
        
        # create pipeline
        model = Pipeline(estimators)
        
        # evaluate pipeline
        scores = cross_val_score(model, Xdata, Ydata, cv=outer_cv, scoring=metrics, n_jobs=-1)
        
        df_res[name] = scores
        print('%s, %s_mean=%0.2f, Time:%0.1f mins' % (name, metrics, scores.mean(), (time.time() - start)/60))
        
    # save results CSV
    # -----------------
    resFile = './results/'+metrics+'-'+targetName+'_'+str(label)+'_'+str(folds)+'-foldCV.csv'
    df_res.to_csv(resFile, index=False)
    
    print('* Scores saved', resFile)
    print('Total time:', (time.time() - total)/60, ' mins')             
    
    plt.figure()
    plt.clf()
    
    meanpointprops = dict(marker='o', markeredgecolor='green',
                          markerfacecolor='green')
    
    color = dict(boxes='black', whiskers='black', medians='blue', caps='red')
    boxplot = df_res.boxplot(column=names,
                             meanprops=meanpointprops, meanline=False, showmeans=True,
                             color=color,
                             whiskerprops = dict(linestyle='-',linewidth=1.0, color='black'),
                             capprops = dict(linestyle='-',linewidth=2.0, color='red')
                            )
                             
    boxplot.set_title('')
    boxplot.set_xlabel('Classifiers')
    boxplot.set_ylabel(metrics)
    boxplot.tick_params(axis='y', which='minor', bottom=True)
    boxplot.yaxis.grid(False)
    boxplot.xaxis.grid(False)
    
    # Remove top and right border
    boxplot.spines['top'].set_visible(False)
    boxplot.spines['right'].set_visible(False)
    
    # Initialize minor ticks
    boxplot.minorticks_on()

    # Now minor ticks exist and are turned on for both axes
    # Turn off x-axis minor ticks
    boxplot.xaxis.set_tick_params(which='minor', bottom=False)
    
    # save the figure as JPEG
    plotFile = './results/'+metrics+'-'+targetName+'_'+str(label)+'_'+str(folds)+'-foldCV.jpeg'
    
    boxplot.figure.savefig(plotFile,format='jpeg', dpi=300)
    
    print('* Saving plot:', plotFile)
    
    # clean each figure
    boxplot.figure.clf()
    
    # return AUC scores for all classifiers as dataframe (each column a classifier)
    return df_res

### Calculations

Set the dataset file as *sFile* and run the pipeline for different classifiers (no feature selection, no scaling):

In [ ]:
# for each subset file
df_results = None # all results 

for OutVar in outVars:
    # define a label for output files
    targetName = 'sel' # for Pool use 'pool'
    sFile = './ds/ds06_selected.txt' # for pool use './ds/ds06_pool.txt'
    
    # METRICS
    metrics= 'roc_auc' # for accuracy use 'accuracy'

    # get data from file
    Xdata, Ydata, Features = getDataFromDataset(sFile,OutVar)

    # Calculate class weights
    class_weights = set_weights(Ydata)
    print("Class weights = ", class_weights)
        
    # try different folds for each subset -> box plots
    for folds in foldTypes:
        
        # calculate outer CV for different binary classifiers
        df_fold = Pipeline_OuterCV(Xdata, Ydata, label = OutVar, class_weights = class_weights, folds = folds, seed = seed, metrics=metrics)
        df_fold['Dataset'] = OutVar
        df_fold['folds'] = folds
        
        # add each result to a summary dataframe
        df_results = pd.concat([df_results,df_fold])

## Box plots from results

Load the results from file (if you dont want to run the previous calculations):

In [ ]:
summaryFile = './results/accuracy-3-foldCV.csv'

print('\n-> Read summary results', summaryFile)
df_results = pd.read_csv(summaryFile)

Get list of classifiers from output file:

In [ ]:
classifierNames = list(df_results.columns)
classifierNames.remove('Dataset')
classifierNames

In [ ]:
import seaborn as sns
dd=pd.melt(df_results,id_vars=['Dataset'],value_vars=classifierNames,var_name='Classifiers')
# replace value 
dd=dd.rename(columns = {'value':'ACC'})
dd

In [ ]:
plt.figure()
plt.clf()
fig, ax = plt.subplots()

sns.set(rc={'figure.figsize':(16,9)})
sns.set_style("whitegrid")
ax = sns.boxplot(x='Classifiers',y='ACC',data=dd,hue='Dataset',showmeans=True,
                meanprops={"marker":"o",
                           "markerfacecolor":"white", #white
                           "markeredgecolor":"black",
                           "markersize":"8"})
#ax.set_ylim(.93, 1.0)
plt.ylabel("ACC", size=14)
plt.xlabel("Machine Learning Classifiers", size=14)

# ax.set(ylim=(0.93, 1.1))
ax.legend(loc='lower right')
plt.savefig('./results/accuracy-3-foldCV_new.jpg', format='jpeg', dpi=300)